# Citations  to an author implemented in new inspire!

In [1]:
import pandas as pd
import time
import sys
import numpy as np
pd.set_option('display.max_colwidth',200)

exact_author='D.Restrepo.1'
url='https://inspirehep.net/api/literature?sort=mostrecent'

df=pd.read_json(
    '{}&size=100&page=1&q=refersto:a:{}'.format(
       url,exact_author))

#control_number: 1849298 → https://inspirehep.net/literature?q=recid:1849298

new=pd.DataFrame( pd.DataFrame( df.loc['hits','hits'] ).metadata.tolist() )

#new.to_json('data/old.json')
old=pd.read_json('data/old.json')

#Return the unique values in `ar1` that are not in `ar2`.
last=np.setdiff1d( new.control_number.to_list(),old.control_number.to_list() )

if len(last)>0:
    new.to_json('data/old.json')

#last=[1849298]

lst=pd.DataFrame()
for l in last:
    lst=lst.append(new[new.control_number==l])

if lst.empty:
    sys.exit('Not new citations today. Ignore next part!')

#Find documents of exact author
size=250
page=1
tm=pd.read_json('{}&size={}&page={}&q=ea+{}'.format(url,size,page,exact_author))
arts=pd.DataFrame( pd.DataFrame( tm.loc['hits','hits'] ).metadata.tolist()  )

total_hits=tm.loc['total','hits']

pages=total_hits//size
if total_hits%size>0:
    pages=pages+1

#Extra documents
for p in range(2,pages+1):
    tm=pd.read_json('{}&size={}&page={}&q=ea+{}'.format(url,size,p,exact_author))
    arts=arts.append( pd.DataFrame( pd.DataFrame( tm.loc['hits','hits'] ).metadata.tolist()  ) 
                    ).reset_index(drop=True)

# Extract citations with exact author
refs=pd.DataFrame()
for i in lst.index:
    refs['refs']=pd.DataFrame( lst.loc[i,'references'] ).record.dropna().apply(lambda d: d.get('$ref').split('https://inspirehep.net/api/literature/')).str[-1].astype(int)

    # Find citations metadata of exact authors
    try:
        cr=arts.merge(refs,left_on='control_number',right_on='refs',how='left'
                     ).dropna(subset=['refs']).apply(
           lambda row: {row['control_number']:row['titles'][0].get('title')},axis='columns'
           ).to_list()
    except:
        cr=[]    

    lst['cited_titles']=None
    lst['cited_titles']=lst['cited_titles'].astype(object)

    lst.at[i,'cited_titles']=cr

#Simplify output
lst['full_names']=lst.authors.apply(lambda l: [d.get('full_name') for d in l])

lst[['arxiv_eprints','titles','full_names','cited_titles']]

SystemExit: Not new citations today. Ignore next part!

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
%%bash
git commit -am 'lh'
git push origin master

## OLD inspire

In [1]:



UPDATE=False

old=pd.read_json('data/old.json')#.reset_index(drop=True)
#old.number_of_citations.sum()

# Download citations of each paper

if UPDATE:
    for r in old['recid']:
        p=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r)
                      ).to_json('data/{}.json'.format(r))
        print(r)
        time.sleep(1)

# Download INSPIRE HEP Profile

# TODO: Check if the author has more than 250 records!

url='http://inspirehep.net/search?p=ea+{}&rg=250&of=recjson'.format(exact_author)
df=pd.read_json(url)
new=df.copy()
#df.number_of_citations.sum()

ANALYSIS=False
if new['number_of_citations'].sum()>old['number_of_citations'].sum():
    ANALYSIS=True
if not ANALYSIS:
    sys.exit('NO NEW CITES TO ANALYSE!')

old['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]
new['iid']=old.FIXME_OAI.apply(lambda d: d['id']).str.split(':').str[-1]

old=old.rename({'number_of_citations':'nc'},axis='columns') ##add other relevant columns for analysis

new=new.merge(old[['nc','iid']],on='iid',how='left')#[['number_of_citations','nc']]

news=new[new['number_of_citations']>new['nc']]

print('New citations in:')
news['recid']
#['primary_report_number'].apply(lambda l:  [x for x in l if x.find('arXiv')>-1])

for r in news['recid']:
    #Analysis of each citation
    print('===== NEW CITATION FOR {}====='.format(r))
    oldrefersto=pd.read_json('data/{}.json'.format(r))
    refersto=pd.read_json('http://inspirehep.net/search?ln=en&p=refersto:recid:{}&of=recjson'.format(r))
    print( news[news['recid']==r]['title'].values[0] )
    print('with')
    print(news[news['recid']==r]['authors'].apply(lambda l: [d['full_name'] for d in l]).values[0])
    print('WAS CITED BY:')
    try:
        newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'])
    except KeyError:
        newrecid=pd.np.setdiff1d(refersto['recid'],[])
    #newrecid=pd.np.setdiff1d(refersto['recid'],oldrefersto['recid'][:-2])
    for rr in newrecid:
        print('*'*60)
        print( refersto[refersto['recid']==rr]['title'].values[0])
        print( refersto[refersto['recid']==rr]['authors'].apply(lambda l: 
                                            [d['full_name'] for d in l]).values[0])
        try:
            doctype=refersto[refersto['recid']==rr]['primary_report_number'].apply(lambda l: 
                                            [x for x in l if x.find('arXiv')>-1]
                                            if type(l)==list else 
                                            [x for x in [l] if x.find('arXiv')>-1 ]).values[0]
            if not doctype: #force TypeError
                doctypetmp=eval(None)#refersto['forceKeyError']
        except (TypeError,AttributeError):
            l=refersto[refersto['recid']==rr].get('collection')
            if not l.empty:
                if 'Thesis' in [d.get('primary') for d in l.values[0]]:
                    doctype='Thesis'
                else:
                    doctype='''WARNING: unkown doctype. Check: refersto[refersto['recid']==rr].get('collection')'''
            else:
                doctype='''WARNING: unkown doctype. Check: refersto[refersto['recid']==rr]'''                    
            
        print( doctype )    
    print('UPDATE CITATIONS (Y)')
    YN=input('Y/N')
    if YN not in ['N','n','not','Not']:
        print('UPDATING JSON DATA CITATION for {}'.format(r))
        refersto.to_json('data/{}.json'.format(r))

New citations in:
===== NEW CITATION FOR 1708907=====
{'title': 'Minimal radiative Dirac neutrino mass models'}
with
['Calle, Julian', 'Restrepo, Diego', 'Yaguna, Carlos E.', 'Zapata, Óscar']
WAS CITED BY:
************************************************************
{'title': 'Leptonic Source of Dark Matter and Radiative Majorana or Dirac Neutrino Mass'}
['Ma, Ernest']
['arXiv:1912.11950']
UPDATE CITATIONS (Y)
Y/Ny
UPDATING JSON DATA CITATION for 1708907
===== NEW CITATION FOR 1679809=====
{'title': 'Bound-state dark matter with Majorana neutrinos'}
with
['Reig, M.', 'Restrepo, D.', 'Valle, J.W. F.', 'Zapata, O.']
WAS CITED BY:
************************************************************
{'title': 'Neutrino physics outlook'}
['Valle, José W.F.']
['arXiv:2001.03016']
UPDATE CITATIONS (Y)
Y/Ny
UPDATING JSON DATA CITATION for 1679809
===== NEW CITATION FOR 1627633=====
{'title': 'Vector Boson Fusion in the Inert Doublet Model'}
with
['Dutta, Bhaskar', 'Palacio, Guillermo', 'Ruiz-Alvarez, 

In [2]:
print('UPDATE FULL DATABASE for profiles (N)')
YN=input('Y/N')
if YN=='Y' or YN=='y' or YN=='yes' or YN=='Yes':
    df.to_json('data/old.json')

UPDATE FULL DATABASE for profiles (N)
Y/Ny


In [3]:
%%bash
git commit -am 'lh'
git push origin master

[master 60fadf8] lh
 20 files changed, 281 insertions(+), 31 deletions(-)
 rewrite data/1222142.json (79%)
 rewrite data/1249404.json (86%)
 rewrite data/1365306.json (83%)
 rewrite data/1607572.json (96%)
 rewrite data/1615477.json (85%)
 rewrite data/1627633.json (91%)
 rewrite data/1679809.json (95%)
 rewrite data/1708907.json (89%)
 rewrite data/468209.json (84%)
 rewrite data/555069.json (86%)
 rewrite data/677884.json (83%)
 rewrite data/770422.json (81%)
 rewrite data/777268.json (85%)
 rewrite data/783258.json (84%)
 rewrite data/793723.json (81%)
 rewrite data/824906.json (89%)
 rewrite data/842933.json (81%)
 rewrite data/859733.json (89%)
 rewrite data/old.json (77%)


To github.com:restrepo/inspire.git
   116e9b4..60fadf8  master -> master
